# Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd

from proforma import prototypes
from proforma.parcels import Parcel
from proforma.run import ModelRun

# Parcels

In [ ]:
def read_parcels():
    df = pd.read_excel('./data/parcels.xlsx')
    return [Parcel(**row.to_dict()) for _, row in df.iterrows()]

In [ ]:
parcels = read_parcels()

# Prototypes

In [ ]:
def read_prototypes():
    PROTOTYPE_DATA = {
        prototypes.OfficePrototype: pd.read_excel('./data/prototypes/office.xlsx'),
        prototypes.RetailPrototype: pd.read_excel('./data/prototypes/retail.xlsx'),
        prototypes.WDPrototype: pd.read_excel('./data/prototypes/wd.xlsx'),
        prototypes.FlexPrototype: pd.read_excel('./data/prototypes/flex.xlsx'),
        prototypes.ResidentialRentalPrototype: pd.read_excel('./data/prototypes/residential_rental.xlsx'),
        prototypes.ResidentialOwnershipPrototype: pd.read_excel('./data/prototypes/residential_ownership.xlsx')
    }
    return [cls(**row.to_dict()) for cls, df in PROTOTYPE_DATA.items() for _, row in df.iterrows()]

In [ ]:
prototypes = read_prototypes()

# Conversion rates

In [ ]:
base_conversion_rates = (
    # Cut-off, rate
    (0.75, 0.00952185618800331),
    (1.25, 0.00515849696448995),
    (2, 0.00186337303278268),
    (4, 0.00050113046153647),
    (pd.np.inf, 0.000617815532037205),
)

# Entitlement screen

In [ ]:
screen = pd.read_excel('./data/entitlement_screen.xlsx', index_col='Zone Class')

# Model Run

In [ ]:
run = ModelRun(parcels, prototypes, base_conversion_rates, screen, n_iterations=5, iteration_length=5)

In [ ]:
df = run.to_df()

# Stats

In [ ]:
# Number of parcels
len(parcels)

In [ ]:
# Run time
%timeit ModelRun(parcels, prototypes, base_conversion_rates, screen, n_iterations=5, iteration_length=5)

In [ ]:
# Run time (50000 parcels)
%timeit ModelRun(parcels * 10, prototypes, base_conversion_rates, screen, n_iterations=5, iteration_length=5)

In [ ]:
# Total acres yielded
df.n_sf.sum() / 43560

In [ ]:
# Total units yielded
df.n_units.sum()

In [ ]:
# Acres by form
df.groupby('prototype_class').n_sf.sum().div(43560)

In [ ]:
# Units by form
df.groupby('prototype_class').n_units.sum()

In [ ]:
# Acres by prototype
df.groupby('prototype').n_sf.sum().div(43560).sort_values(ascending=False)

In [ ]:
# Acres by prototype
df.groupby('prototype').n_units.sum().sort_values(ascending=False)